In [12]:
import requests
import pandas as pd
from datetime import *
# from airflow import *
# from airflow.operators.python import PythonOperator
# from airflow.utils.dates import days_ago
# from airflow.hooks.base import BaseHook
from snowflake.connector.pandas_tools import write_pandas
import snowflake.connector



In [13]:
url = 'https://api.weatherapi.com/v1/current.json?key=98078c8de2274791b03161315240410&q=London&aqi=yes'

In [26]:
# function to retrieve raw data and extract location and temperature
def get_data():
    data = requests.get(url).json()
    df = pd.json_normalize(data)

    # location_name = data['location']['name']
    # temp_c = data['current']['temp_c']

    # df = pd.DataFrame({
    #     'name': [location_name],
    #     'temp_c': [temp_c]
    #})
    return df

raw_data = get_data()
print(type(raw_data))

<class 'pandas.core.frame.DataFrame'>


In [27]:
# Connection parameters
user = 'PROJECTSMITH'          # Your Snowflake username
password = 'Testpassword95'  # Your Snowflake password
account = 'ISOCBXH-ER59203'    # Snowflake account name (e.g., 'account.region')
warehouse = 'COMPUTE_WH'  # The warehouse to use
database = 'WEATHERAPI'  # The database to connect to
schema = 'RAW'      # The schema to use


In [28]:
snowflake_connection = snowflake.connector.connect(
    user='PROJECTSMITH',
    password='Testpassword95',
    account='ISOCBXH-ER59203',
    warehouse='COMPUTE_WH',
    database='WEATHERAPI',
    schema='RAW',
    role='ACCOUNTADMIN'
)

In [30]:
# writing the table into snowflake
success, nchunks, nrows, _ = write_pandas(
                conn = snowflake_connection,
                df = raw_data,
                table_name='temperature',  # The raw data table
                database=database,
                schema=schema,
                auto_create_table=True,
                overwrite=False  # Ensure we're appending, not overwriting
                
            )